In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 40)

# Use this to see how much memory the dataframes use
from sys import getsizeof

from vivarium_research_prl.noise import corruption, fake_names, noisify
from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Mon 06 Mar 2023 11:19:31 AM PST
ndbs
Linux int-slurm-sarchive-p0002 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/linkage/wic_case_study/v2.0


In [2]:
%load_ext autoreload
%autoreload 2

# Function to get the size of an object in megabytes

In [37]:
def sizemb(obj):
    return getsizeof(obj) / 1e6

# Find and load data

From Jim posted by Jim in Slack on Tue 2/28/2023:
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/vv_mailing_addresses/united_states_of_america/2023_02_27_10_25_20/final_results/2023_02_27_14_59_44
```

## First load just a few rows to check out dtypes, etc.

In [3]:
project_output_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/vv_mailing_addresses'
    '/united_states_of_america/2023_02_27_10_25_20'
    '/final_results/2023_02_27_14_59_44'
)
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -halt $output_dir

total 7.4G
drwxrwsr-x 3 albrja IHME-Simulationscience 1.0K Feb 27 17:50 ..
-rw-r--r-- 1 albrja IHME-Simulationscience 6.4G Feb 27 17:49 tax_w2_observer.csv.bz2
drwxrwsr-x 2 albrja IHME-Simulationscience 3.0K Feb 27 16:46 .
-rw-r--r-- 1 albrja IHME-Simulationscience 141M Feb 27 16:37 social_security_observer.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience  71M Feb 27 16:35 wic_observer.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience  40M Feb 27 16:34 household_survey_observer_cps.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience 7.9M Feb 27 16:33 household_survey_observer_acs.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience 752M Feb 27 16:33 decennial_census_observer.csv.bz2


In [4]:
df_census_rows = pd.read_csv(f'{output_dir}/decennial_census_observer.csv.bz2', nrows=1000)
df_census_rows

,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254_0,3254_0,3254_0,3254_0,59.550114,Male,White,1960-09-12,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254_1,3254_1,3254_1,3254_0,58.674887,Female,White,1961-07-29,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254_2,3254_2,3254_2,3254_2,53.272919,Male,White,1966-12-23,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254_3,3254_3,3254_3,3254_2,54.643021,Female,White,1965-08-10,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254_4,3254_4,3254_4,3254_4,55.642824,Female,Black,1964-08-09,3254_8,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3254_995,3254_995,3254_995,3254_995,83.518744,Female,White,1936-09-24,3254_408,4,...,old turtle pond road,NaN,tucson,33264,old turtle pond road,NaN,0,AZ,tucson,85382
996,3254_996,3254_996,3254_996,3254_995,62.464713,Female,White,1957-10-14,3254_408,4,...,old turtle pond road,NaN,tucson,33264,old turtle pond road,NaN,0,AZ,tucson,85382
997,3254_997,3254_997,3254_997,3254_997,24.819289,Female,Black,1995-06-07,3254_409,17,...,webster st,NaN,montgomery,30641,webster st,NaN,0,IL,montgomery,60083
998,3254_998,3254_998,3254_998,3254_998,28.565089,Male,White,1991-09-08,3254_410,26,...,hill lakes ct,NaN,detroit,20,hill lakes ct,NaN,0,MI,detroit,48359


In [5]:
df_census_rows.dtypes

simulant_id                       object
first_name_id                     object
middle_name_id                    object
last_name_id                      object
age                              float64
sex                               object
race_ethnicity                    object
date_of_birth                     object
address_id                        object
state_id                           int64
po_box                             int64
puma                               int64
guardian_1                         int64
guardian_2                         int64
guardian_1_address_id            float64
guardian_2_address_id            float64
relation_to_household_head        object
census_year                        int64
housing_type                      object
random_seed                        int64
year_of_birth                      int64
state                             object
first_name                        object
middle_initial                    object
last_name       

In [7]:
df_census_rows.dtypes.value_counts()

object     23
int64      11
float64     3
dtype: int64

In [8]:
df_census_rows.dtypes.loc[df_census_rows.dtypes=='float64']

age                      float64
guardian_1_address_id    float64
guardian_2_address_id    float64
dtype: object

In [18]:
int_cols = df_census_rows.dtypes.loc[df_census_rows.dtypes=='int64']
int_cols

state_id                   int64
po_box                     int64
puma                       int64
guardian_1                 int64
guardian_2                 int64
census_year                int64
random_seed                int64
year_of_birth              int64
zipcode                    int64
mailing_address_po_box     int64
mailing_address_zipcode    int64
dtype: object

In [16]:
object_cols = df_census_rows.dtypes.loc[df_census_rows.dtypes=='object']
object_cols

simulant_id                      object
first_name_id                    object
middle_name_id                   object
last_name_id                     object
sex                              object
race_ethnicity                   object
date_of_birth                    object
address_id                       object
relation_to_household_head       object
housing_type                     object
state                            object
first_name                       object
middle_initial                   object
last_name                        object
street_number                    object
street_name                      object
unit_number                      object
city                             object
mailing_address_street_number    object
mailing_address_street_name      object
mailing_address_unit_number      object
mailing_address_state            object
mailing_address_city             object
dtype: object

In [14]:
df_census_rows.filter(regex=r'guardian_\d_address').value_counts(dropna=False)

guardian_1_address_id  guardian_2_address_id
NaN                    NaN                      792
261.0                  NaN                        5
12.0                   12.0                       4
294380.0               NaN                        4
293.0                  NaN                        4
                                               ... 
43.0                   NaN                        1
268.0                  NaN                        1
275.0                  NaN                        1
279.0                  NaN                        1
255.0                  255.0                      1
Length: 118, dtype: int64

In [15]:
df_census_rows.dtypes.loc[df_census_rows.dtypes=='object'].index

Index(['simulant_id', 'first_name_id', 'middle_name_id', 'last_name_id', 'sex',
       'race_ethnicity', 'date_of_birth', 'address_id',
       'relation_to_household_head', 'housing_type', 'state', 'first_name',
       'middle_initial', 'last_name', 'street_number', 'street_name',
       'unit_number', 'city', 'mailing_address_street_number',
       'mailing_address_street_name', 'mailing_address_unit_number',
       'mailing_address_state', 'mailing_address_city'],
      dtype='object')

In [17]:
object_cols.filter(like='_id')

simulant_id       object
first_name_id     object
middle_name_id    object
last_name_id      object
address_id        object
dtype: object

In [21]:
int_cols.index

Index(['state_id', 'po_box', 'puma', 'guardian_1', 'guardian_2', 'census_year',
       'random_seed', 'year_of_birth', 'zipcode', 'mailing_address_po_box',
       'mailing_address_zipcode'],
      dtype='object')

In [23]:
df_census_rows.puma.nunique()

256

In [24]:
df_census_rows.state_id.unique()

array([25, 55, 24, 12,  6, 48, 27, 41, 36, 13, 39, 21, 26, 47, 28, 17,  9,
       34,  8, 51, 54, 19, 37, 45, 42,  5, 49, 46, 15, 50,  4, 23, 22, 18,
       16,  1, 35, 29, 40, 32, 33, 53, 31, 20,  2, 38, 10])

# Now read in full decennial census data with my new `load_data` function based on code from my previous notebook

In [25]:
%%time
df_census_raw = datasets.load_data(f'{output_dir}/decennial_census_observer.csv.bz2')
df_census_raw

CPU times: user 5min 4s, sys: 12.7 s, total: 5min 17s
Wall time: 5min 19s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254_0,3254_0,3254_0,3254_0,59.550114,Male,White,1960-09-12,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254_1,3254_1,3254_1,3254_0,58.674887,Female,White,1961-07-29,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254_2,3254_2,3254_2,3254_2,53.272919,Male,White,1966-12-23,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254_3,3254_3,3254_3,3254_2,54.643021,Female,White,1965-08-10,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254_4,3254_4,3254_4,3254_4,55.642824,Female,Black,1964-08-09,3254_8,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15448912,3541_883506,3541_883506,3541_883506,3541_883504,7.739110,Male,White,2022-07-21,3541_618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448913,3541_883507,3541_883507,3541_883507,3541_883504,31.564120,Female,White,1998-09-23,3541_618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448914,3541_883508,3541_883508,3541_883508,3541_883508,27.369618,Male,Latino,2002-12-03,3541_618379,37,...,old silo rd,NaN,mooresville,1780,old silo rd,NaN,0,NC,mooresville,27217
15448915,3541_883509,3541_883509,3541_883509,3541_883509,71.134048,Male,Asian,1959-02-27,3541_618380,6,...,blue rdge ave,NaN,visalia,565,blue rdge ave,NaN,0,CA,visalia,91770


In [26]:
getsizeof(df_census_raw)

8828794338

In [27]:
%%time
getsizeof(df_census_raw) / 1e6

CPU times: user 7.6 s, sys: 3.73 ms, total: 7.61 s
Wall time: 7.59 s


8828.794338

# See how big the file is when writing it to an HDF file with `bzip2` compression

complevel can be 0 - 9, so try 5 for "medium" compression. Presumably higher numbers lead to more compression while lower numbers lead to faster read/write.

By switching most columns to Categorical and writing to HDF with bzip2 compression, the file size is reduced to 634M from 752M for the compressed .csv.

In [29]:
%%time
df_census_raw.to_hdf(
    'census_categoical_1.hdf', 'decennial_census',
    format='table',
    complevel=5, complib='bzip2'
)

!ls -halt

total 634M
-rw-rw-r-- 1 ndbs IHME-users 634M Mar  6 17:09 census_categoical_1.hdf
drwxrwsr-x 3 ndbs IHME-users 2.0K Mar  6 17:09 .
-rw-rw-r-- 1 ndbs IHME-users  48K Mar  6 17:05 2023_03_06_generate_wic_and_census_data.ipynb
drwxrwsr-x 5 ndbs IHME-users 6.5K Mar  6 11:30 ..
drwxrwsr-x 2 ndbs IHME-users 1.0K Mar  6 11:19 .ipynb_checkpoints
-rw-rw-r-- 1 ndbs IHME-users 1.5K Mar  3 16:14 2023_03_03b_prep_wic_and_census_for_splink.ipynb
CPU times: user 4min 10s, sys: 5.06 s, total: 4min 16s
Wall time: 4min 16s


# The time to read in the compressed HDF file is faster than reading in the compressed csv

In my previous notebook, it took 3min 28s to read in the .csv.bz2. The .hdf below took 2min 10s. 

In [30]:
%%time
df_census_from_hdf = pd.read_hdf('census_categoical_1.hdf')
df_census_from_hdf

CPU times: user 2min 2s, sys: 8.2 s, total: 2min 10s
Wall time: 2min 10s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254_0,3254_0,3254_0,3254_0,59.550114,Male,White,1960-09-12,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254_1,3254_1,3254_1,3254_0,58.674887,Female,White,1961-07-29,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254_2,3254_2,3254_2,3254_2,53.272919,Male,White,1966-12-23,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254_3,3254_3,3254_3,3254_2,54.643021,Female,White,1965-08-10,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254_4,3254_4,3254_4,3254_4,55.642824,Female,Black,1964-08-09,3254_8,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15448912,3541_883506,3541_883506,3541_883506,3541_883504,7.739110,Male,White,2022-07-21,3541_618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448913,3541_883507,3541_883507,3541_883507,3541_883504,31.564120,Female,White,1998-09-23,3541_618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448914,3541_883508,3541_883508,3541_883508,3541_883508,27.369618,Male,Latino,2002-12-03,3541_618379,37,...,old silo rd,NaN,mooresville,1780,old silo rd,NaN,0,NC,mooresville,27217
15448915,3541_883509,3541_883509,3541_883509,3541_883509,71.134048,Male,Asian,1959-02-27,3541_618380,6,...,blue rdge ave,NaN,visalia,565,blue rdge ave,NaN,0,CA,visalia,91770


In [31]:
%%time
getsizeof(df_census_from_hdf) / 1e6

CPU times: user 12.5 s, sys: 84.3 ms, total: 12.6 s
Wall time: 12.6 s


8952.340543

In [33]:
%%time
df_census_from_hdf.equals(df_census_raw)

CPU times: user 14.2 s, sys: 264 ms, total: 14.5 s
Wall time: 14.5 s


True

In [35]:
df_census_raw.random_seed.nunique()

10

In [38]:
sizemb(df_census_raw.random_seed)

15.449971

In [40]:
sizemb(df_census_raw.random_seed.astype('int8'))

15.449061

In [41]:
df_census_raw.random_seed

0           3254
1           3254
2           3254
3           3254
4           3254
            ... 
15448912    3541
15448913    3541
15448914    3541
15448915    3541
15448916    3541
Name: random_seed, Length: 15448917, dtype: category
Categories (10, object): ['3254', '4344', '6810', '2787', ..., '4369', '7359', '2284', '3541']

In [43]:
sizemb(df_census_raw.state_id)

15.454167

In [44]:
sizemb(df_census_raw.state_id.astype('int8'))

15.449061

In [45]:
sizemb(df_census_raw.state)

911.486247

In [46]:
df_census_raw.state

0           MA
1           MA
2           WI
3           WI
4           MD
            ..
15448912    CA
15448913    CA
15448914    NC
15448915    CA
15448916    PA
Name: state, Length: 15448917, dtype: object

In [47]:
df_census_raw.dtypes

simulant_id                        object
first_name_id                      object
middle_name_id                     object
last_name_id                       object
age                               float64
sex                              category
race_ethnicity                   category
date_of_birth                    category
address_id                         object
state_id                         category
po_box                           category
puma                             category
guardian_1                          int64
guardian_2                          int64
guardian_1_address_id             float64
guardian_2_address_id             float64
relation_to_household_head       category
census_year                      category
housing_type                     category
random_seed                      category
year_of_birth                    category
state                              object
first_name                       category
middle_initial                    

In [49]:
sizemb(pd.Categorical(df_census_raw.state))

15.454046

In [50]:
sizemb(pd.Categorical(df_census_raw.middle_initial))

15.451513

In [51]:
sizemb(df_census_raw.middle_initial)

896.03733

In [52]:
df_census_raw.guardian_1.nunique()

704932

In [55]:
print(2**16, 2**32)

65536 4294967296


In [56]:
sizemb(df_census_raw.guardian_1)

123.59148

In [57]:
getsizeof(df_census_raw.guardian_1) / len(df_census_raw)

8.000009321041727

In [58]:
sizemb(pd.Categorical(df_census_raw.guardian_1))

84.343468

In [59]:
sizemb(df_census_raw.guardian_1.astype('int32'))

61.795812

In [60]:
getsizeof(df_census_raw.guardian_1.astype('int32')) / len(df_census_raw)

4.000009321041728

# Now try converting string IDs to ints

In [64]:
column_cats = datasets.get_column_categories()
column_cats

{'string_ids': ['simulant_id',
  'first_name_id',
  'middle_name_id',
  'last_name_id',
  'address_id'],
 'categorical': ['sex',
  'race_ethnicity',
  'relation_to_household_head',
  'housing_type',
  'state',
  'middle_initial',
  'year_of_birth',
  'census_year',
  'random_seed',
  'state_id',
  'puma'],
 'optional_categorical': ['date_of_birth',
  'first_name',
  'last_name',
  'street_number',
  'street_name',
  'unit_number',
  'city',
  'mailing_address_street_number',
  'mailing_address_street_name',
  'mailing_address_unit_number',
  'mailing_address_state',
  'mailing_address_city',
  'zipcode',
  'mailing_address_zipcode',
  'po_box',
  'mailing_address_po_box'],
 'int_ids': ['guardian_1', 'guardian_2'],
 'float_ids': ['guardian_1_address_id', 'guardian_2_address_id'],
 'float_cols': ['age']}

In [66]:
%%time
datasets.convert_string_id_cols(df_census_raw)
df_census_raw.dtypes

CPU times: user 2min 50s, sys: 11.6 s, total: 3min 2s
Wall time: 3min 2s


simulant_id                         int64
first_name_id                       int64
middle_name_id                      int64
last_name_id                        int64
age                               float64
sex                              category
race_ethnicity                   category
date_of_birth                    category
address_id                          int64
state_id                         category
po_box                           category
puma                             category
guardian_1                          int64
guardian_2                          int64
guardian_1_address_id             float64
guardian_2_address_id             float64
relation_to_household_head       category
census_year                      category
housing_type                     category
random_seed                      category
year_of_birth                    category
state                              object
first_name                       category
middle_initial                    

In [67]:
%%time
sizemb(df_census_raw)

CPU times: user 2.5 s, sys: 252 ms, total: 2.75 s
Wall time: 2.74 s


4210.923283

In [68]:
df_census_raw.filter(like='_id')

,simulant_id,first_name_id,middle_name_id,last_name_id,address_id,state_id,guardian_1_address_id,guardian_2_address_id
0,3254000000000,3254000000000,3254000000000,3254000000000,3254000000006,25,NaN,NaN
1,3254000000001,3254000000001,3254000000001,3254000000000,3254000000006,25,NaN,NaN
2,3254000000002,3254000000002,3254000000002,3254000000002,3254000000007,55,NaN,NaN
3,3254000000003,3254000000003,3254000000003,3254000000002,3254000000007,55,NaN,NaN
4,3254000000004,3254000000004,3254000000004,3254000000004,3254000000008,24,NaN,NaN
...,...,...,...,...,...,...,...,...
15448912,3541000883506,3541000883506,3541000883506,3541000883504,3541000618378,6,618378.0,618378.0
15448913,3541000883507,3541000883507,3541000883507,3541000883504,3541000618378,6,NaN,NaN
15448914,3541000883508,3541000883508,3541000883508,3541000883508,3541000618379,37,NaN,NaN
15448915,3541000883509,3541000883509,3541000883509,3541000883509,3541000618380,6,NaN,NaN


# Convert `state` and `middle_initial` to categorical -- I missed them originally because of a missing comma

In [69]:
%%time
for col in ['state', 'middle_initial']:
    df_census_raw[col] = pd.Categorical(df_census_raw[col])
df_census_raw.dtypes

CPU times: user 1.22 s, sys: 324 ms, total: 1.54 s
Wall time: 1.54 s


simulant_id                         int64
first_name_id                       int64
middle_name_id                      int64
last_name_id                        int64
age                               float64
sex                              category
race_ethnicity                   category
date_of_birth                    category
address_id                          int64
state_id                         category
po_box                           category
puma                             category
guardian_1                          int64
guardian_2                          int64
guardian_1_address_id             float64
guardian_2_address_id             float64
relation_to_household_head       category
census_year                      category
housing_type                     category
random_seed                      category
year_of_birth                    category
state                            category
first_name                       category
middle_initial                   c

In [70]:
sizemb(df_census_raw)

2434.305521

In [71]:
%%time
df_census_raw.to_hdf(
    'census_categoical_and_int_1.hdf', 'decennial_census',
    format='table',
    complevel=5, complib='bzip2'
)

!ls -halt

total 1.3G
-rw-rw-r-- 1 ndbs IHME-users 615M Mar  6 23:07 census_categoical_and_int_1.hdf
drwxrwsr-x 3 ndbs IHME-users 2.5K Mar  6 23:07 .
-rw-rw-r-- 1 ndbs IHME-users  91K Mar  6 23:06 2023_03_06_generate_wic_and_census_data.ipynb
-rw-rw-r-- 1 ndbs IHME-users 634M Mar  6 17:09 census_categoical_1.hdf
drwxrwsr-x 5 ndbs IHME-users 6.5K Mar  6 11:30 ..
drwxrwsr-x 2 ndbs IHME-users 1.0K Mar  6 11:19 .ipynb_checkpoints
-rw-rw-r-- 1 ndbs IHME-users 1.5K Mar  3 16:14 2023_03_03b_prep_wic_and_census_for_splink.ipynb
CPU times: user 2min 59s, sys: 1.71 s, total: 3min
Wall time: 3min 2s


In [72]:
%%time
df_census_from_hdf = pd.read_hdf('census_categoical_and_int_1.hdf')
df_census_from_hdf

CPU times: user 1min 11s, sys: 2.82 s, total: 1min 14s
Wall time: 1min 14s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254000000000,3254000000000,3254000000000,3254000000000,59.550114,Male,White,1960-09-12,3254000000006,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254000000001,3254000000001,3254000000001,3254000000000,58.674887,Female,White,1961-07-29,3254000000006,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254000000002,3254000000002,3254000000002,3254000000002,53.272919,Male,White,1966-12-23,3254000000007,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254000000003,3254000000003,3254000000003,3254000000002,54.643021,Female,White,1965-08-10,3254000000007,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254000000004,3254000000004,3254000000004,3254000000004,55.642824,Female,Black,1964-08-09,3254000000008,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15448912,3541000883506,3541000883506,3541000883506,3541000883504,7.739110,Male,White,2022-07-21,3541000618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448913,3541000883507,3541000883507,3541000883507,3541000883504,31.564120,Female,White,1998-09-23,3541000618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448914,3541000883508,3541000883508,3541000883508,3541000883508,27.369618,Male,Latino,2002-12-03,3541000618379,37,...,old silo rd,NaN,mooresville,1780,old silo rd,NaN,0,NC,mooresville,27217
15448915,3541000883509,3541000883509,3541000883509,3541000883509,71.134048,Male,Asian,1959-02-27,3541000618380,6,...,blue rdge ave,NaN,visalia,565,blue rdge ave,NaN,0,CA,visalia,91770


In [73]:
%%time
df_census_from_hdf.equals(df_census_raw)

CPU times: user 2.08 s, sys: 413 ms, total: 2.49 s
Wall time: 2.49 s


True

In [74]:
df_census_raw.zipcode

0            2740
1            2740
2           53952
3           53952
4           21239
            ...  
15448912    95124
15448913    95124
15448914    27217
15448915    91770
15448916    19081
Name: zipcode, Length: 15448917, dtype: category
Categories (32134, object): ['10003', '10009', '1001', '10010', ..., '60969', '69036', '97911', '71998']

In [75]:
df_census_raw.zipcode.nunique()

32134

In [76]:
df_census_raw.zipcode[0]

'2740'

In [77]:
type(df_census_raw.zipcode[0])

str

In [80]:
df_census_raw.zipcode.str.zfill(5)

0           02740
1           02740
2           53952
3           53952
4           21239
            ...  
15448912    95124
15448913    95124
15448914    27217
15448915    91770
15448916    19081
Name: zipcode, Length: 15448917, dtype: object

In [81]:
df_census_raw.zipcode.str.isdigit().all()

True

In [78]:
df_census_rows = pd.read_csv(
    f'{output_dir}/decennial_census_observer.csv.bz2',
    nrows=1000,
    dtype={'zipcode': str}
)
df_census_rows

,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254_0,3254_0,3254_0,3254_0,59.550114,Male,White,1960-09-12,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254_1,3254_1,3254_1,3254_0,58.674887,Female,White,1961-07-29,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254_2,3254_2,3254_2,3254_2,53.272919,Male,White,1966-12-23,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254_3,3254_3,3254_3,3254_2,54.643021,Female,White,1965-08-10,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254_4,3254_4,3254_4,3254_4,55.642824,Female,Black,1964-08-09,3254_8,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3254_995,3254_995,3254_995,3254_995,83.518744,Female,White,1936-09-24,3254_408,4,...,old turtle pond road,NaN,tucson,33264,old turtle pond road,NaN,0,AZ,tucson,85382
996,3254_996,3254_996,3254_996,3254_995,62.464713,Female,White,1957-10-14,3254_408,4,...,old turtle pond road,NaN,tucson,33264,old turtle pond road,NaN,0,AZ,tucson,85382
997,3254_997,3254_997,3254_997,3254_997,24.819289,Female,Black,1995-06-07,3254_409,17,...,webster st,NaN,montgomery,30641,webster st,NaN,0,IL,montgomery,60083
998,3254_998,3254_998,3254_998,3254_998,28.565089,Male,White,1991-09-08,3254_410,26,...,hill lakes ct,NaN,detroit,20,hill lakes ct,NaN,0,MI,detroit,48359


In [79]:
df_census_rows.zipcode[0]

'2740'

In [88]:
a = np.array(np.random.rand(2000), dtype='float16')
a

array([0.998 , 0.771 , 0.656 , ..., 0.3638, 0.7856, 0.814 ], dtype=float16)

In [89]:
a[0]

0.998

In [90]:
df_census_raw.guardian_1

0               -1
1               -1
2               -1
3               -1
4               -1
             ...  
15448912    883504
15448913        -1
15448914        -1
15448915        -1
15448916        -1
Name: guardian_1, Length: 15448917, dtype: int64

In [91]:
df_census_raw.guardian_1.astype('float16')

0          -1.0
1          -1.0
2          -1.0
3          -1.0
4          -1.0
           ... 
15448912    inf
15448913   -1.0
15448914   -1.0
15448915   -1.0
15448916   -1.0
Name: guardian_1, Length: 15448917, dtype: float16

# Try reloading census with updated load function

In [96]:
%%time
df_census_raw = datasets.load_data(
    f'{output_dir}/decennial_census_observer.csv.bz2',
    convert_str_ids=True
)
df_census_raw

CPU times: user 9min 27s, sys: 21.1 s, total: 9min 48s
Wall time: 9min 49s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254000000000,3254000000000,3254000000000,3254000000000,59.550114,Male,White,1960-09-12,3254000000006,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254000000001,3254000000001,3254000000001,3254000000000,58.674887,Female,White,1961-07-29,3254000000006,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254000000002,3254000000002,3254000000002,3254000000002,53.272919,Male,White,1966-12-23,3254000000007,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254000000003,3254000000003,3254000000003,3254000000002,54.643021,Female,White,1965-08-10,3254000000007,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254000000004,3254000000004,3254000000004,3254000000004,55.642824,Female,Black,1964-08-09,3254000000008,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15448912,3541000883506,3541000883506,3541000883506,3541000883504,7.739110,Male,White,2022-07-21,3541000618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448913,3541000883507,3541000883507,3541000883507,3541000883504,31.564120,Female,White,1998-09-23,3541000618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448914,3541000883508,3541000883508,3541000883508,3541000883508,27.369618,Male,Latino,2002-12-03,3541000618379,37,...,old silo rd,NaN,mooresville,1780,old silo rd,NaN,0,NC,mooresville,27217
15448915,3541000883509,3541000883509,3541000883509,3541000883509,71.134048,Male,Asian,1959-02-27,3541000618380,6,...,blue rdge ave,NaN,visalia,565,blue rdge ave,NaN,0,CA,visalia,91770


In [97]:
sizemb(df_census_raw)

2434.305521

In [98]:
df_census_raw.dtypes

simulant_id                         int64
first_name_id                       int64
middle_name_id                      int64
last_name_id                        int64
age                               float64
sex                              category
race_ethnicity                   category
date_of_birth                    category
address_id                          int64
state_id                         category
po_box                           category
puma                             category
guardian_1                          int64
guardian_2                          int64
guardian_1_address_id             float64
guardian_2_address_id             float64
relation_to_household_head       category
census_year                      category
housing_type                     category
random_seed                      category
year_of_birth                    category
state                            category
first_name                       category
middle_initial                   c

In [99]:
df_census_raw.age

0           59.550114
1           58.674887
2           53.272919
3           54.643021
4           55.642824
              ...    
15448912     7.739110
15448913    31.564120
15448914    27.369618
15448915    71.134048
15448916    25.086591
Name: age, Length: 15448917, dtype: float64

In [100]:
df_census_raw.age.astype('float16')

0           59.562500
1           58.687500
2           53.281250
3           54.656250
4           55.656250
              ...    
15448912     7.738281
15448913    31.562500
15448914    27.375000
15448915    71.125000
15448916    25.093750
Name: age, Length: 15448917, dtype: float16

In [101]:
df_census_raw.age.astype('float16').nunique()

14180

In [102]:
df_census_raw.age.astype('float32').nunique()

11764318

In [103]:
df_census_raw.age.nunique()

15385066

In [104]:
df_census_raw.age.astype('float32')

0           59.550114
1           58.674889
2           53.272919
3           54.643021
4           55.642822
              ...    
15448912     7.739110
15448913    31.564119
15448914    27.369617
15448915    71.134048
15448916    25.086592
Name: age, Length: 15448917, dtype: float32

In [106]:
df_census_raw.puma.nunique()

982

In [107]:
getsizeof(df_census_raw.puma) / len(df_census_raw)

2.006029419408493

In [108]:
sizemb(df_census_raw.puma)

30.990982

In [110]:
sizemb(df_census_raw.puma.astype('int16'))

30.897978

# Test reversing a dictionary of lists

In [114]:
d = {'w': ['a'], 'x': [], 'y': ['z', 'q']}
{col: key for key, val in d.items() for col in val}

{'a': 'w', 'z': 'y', 'q': 'y'}

# Test passing `dtype={}` to `read_csv` instead of the default `dtype=None`

It looks like it's equivalent, which is convenient for avoiding having to code a special case.

In [115]:
df_census_rows2 = pd.read_csv(
    f'{output_dir}/decennial_census_observer.csv.bz2',
    nrows=1000,
    dtype={},
)
df_census_rows2

,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254_0,3254_0,3254_0,3254_0,59.550114,Male,White,1960-09-12,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254_1,3254_1,3254_1,3254_0,58.674887,Female,White,1961-07-29,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254_2,3254_2,3254_2,3254_2,53.272919,Male,White,1966-12-23,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254_3,3254_3,3254_3,3254_2,54.643021,Female,White,1965-08-10,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254_4,3254_4,3254_4,3254_4,55.642824,Female,Black,1964-08-09,3254_8,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3254_995,3254_995,3254_995,3254_995,83.518744,Female,White,1936-09-24,3254_408,4,...,old turtle pond road,NaN,tucson,33264,old turtle pond road,NaN,0,AZ,tucson,85382
996,3254_996,3254_996,3254_996,3254_995,62.464713,Female,White,1957-10-14,3254_408,4,...,old turtle pond road,NaN,tucson,33264,old turtle pond road,NaN,0,AZ,tucson,85382
997,3254_997,3254_997,3254_997,3254_997,24.819289,Female,Black,1995-06-07,3254_409,17,...,webster st,NaN,montgomery,30641,webster st,NaN,0,IL,montgomery,60083
998,3254_998,3254_998,3254_998,3254_998,28.565089,Male,White,1991-09-08,3254_410,26,...,hill lakes ct,NaN,detroit,20,hill lakes ct,NaN,0,MI,detroit,48359


In [117]:
df_census_rows2.dtypes

simulant_id                       object
first_name_id                     object
middle_name_id                    object
last_name_id                      object
age                              float64
sex                               object
race_ethnicity                    object
date_of_birth                     object
address_id                        object
state_id                           int64
po_box                             int64
puma                               int64
guardian_1                         int64
guardian_2                         int64
guardian_1_address_id            float64
guardian_2_address_id            float64
relation_to_household_head        object
census_year                        int64
housing_type                      object
random_seed                        int64
year_of_birth                      int64
state                             object
first_name                        object
middle_initial                    object
last_name       

# Try one more time after updates and refactoring

The new version is 2.13G in memory vs. 23.8G for reading in the original csv without specifying dtypes or converting the string ids to integers.

In [120]:
%%time
df_census_raw = datasets.load_data(
    f'{output_dir}/decennial_census_observer.csv.bz2',
    convert_str_ids=True
)
df_census_raw

CPU times: user 9min 1s, sys: 20.4 s, total: 9min 22s
Wall time: 9min 22s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254000000000,3254000000000,3254000000000,3254000000000,59.550114,Male,White,1960-09-12,3254000000006,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254000000001,3254000000001,3254000000001,3254000000000,58.674889,Female,White,1961-07-29,3254000000006,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254000000002,3254000000002,3254000000002,3254000000002,53.272919,Male,White,1966-12-23,3254000000007,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254000000003,3254000000003,3254000000003,3254000000002,54.643021,Female,White,1965-08-10,3254000000007,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254000000004,3254000000004,3254000000004,3254000000004,55.642822,Female,Black,1964-08-09,3254000000008,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15448912,3541000883506,3541000883506,3541000883506,3541000883504,7.739110,Male,White,2022-07-21,3541000618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448913,3541000883507,3541000883507,3541000883507,3541000883504,31.564119,Female,White,1998-09-23,3541000618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448914,3541000883508,3541000883508,3541000883508,3541000883508,27.369617,Male,Latino,2002-12-03,3541000618379,37,...,old silo rd,NaN,mooresville,1780,old silo rd,NaN,0,NC,mooresville,27217
15448915,3541000883509,3541000883509,3541000883509,3541000883509,71.134048,Male,Asian,1959-02-27,3541000618380,6,...,blue rdge ave,NaN,visalia,565,blue rdge ave,NaN,0,CA,visalia,91770


In [121]:
%%time
sizemb(df_census_raw)

CPU times: user 524 ms, sys: 27.8 ms, total: 552 ms
Wall time: 550 ms


2125.228161

In [122]:
df_census_raw.dtypes

simulant_id                         int64
first_name_id                       int64
middle_name_id                      int64
last_name_id                        int64
age                               float32
sex                              category
race_ethnicity                   category
date_of_birth                    category
address_id                          int64
state_id                             int8
po_box                           category
puma                                int16
guardian_1                          int32
guardian_2                          int32
guardian_1_address_id             float32
guardian_2_address_id             float32
relation_to_household_head       category
census_year                      category
housing_type                     category
random_seed                          int8
year_of_birth                    category
state                            category
first_name                       category
middle_initial                   c

# Try saving dtype-optimized version to HDF

The bzip2-compressed HDF only uses 554M on disk vs. 752M for the original csv.bz2, and it takes less than 3 min to save the HDF with level 5 compression.

In [123]:
%%time
df_census_raw.to_hdf(
    'census_categoical_and_int_2.hdf', 'decennial_census',
    format='table',
    complevel=5, complib='bzip2'
)

!ls -halt

total 1.8G
-rw-rw-r-- 1 ndbs IHME-users 554M Mar  7 15:43 census_categoical_and_int_2.hdf
drwxrwsr-x 3 ndbs IHME-users 3.0K Mar  7 15:43 .
-rw-rw-r-- 1 ndbs IHME-users 195K Mar  7 15:42 2023_03_06_generate_wic_and_census_data.ipynb
-rw-rw-r-- 1 ndbs IHME-users 615M Mar  6 23:07 census_categoical_and_int_1.hdf
-rw-rw-r-- 1 ndbs IHME-users 634M Mar  6 17:09 census_categoical_1.hdf
drwxrwsr-x 5 ndbs IHME-users 6.5K Mar  6 11:30 ..
drwxrwsr-x 2 ndbs IHME-users 1.0K Mar  6 11:19 .ipynb_checkpoints
-rw-rw-r-- 1 ndbs IHME-users 1.5K Mar  3 16:14 2023_03_03b_prep_wic_and_census_for_splink.ipynb
CPU times: user 2min 44s, sys: 2.32 s, total: 2min 46s
Wall time: 2min 47s


# Reading in the compressed HDF only takes 1min 12s

The original .csv.bz2 took 3min 28s to load in my previous notebook.

In [124]:
%%time
df_census_from_hdf = pd.read_hdf('census_categoical_and_int_2.hdf')
df_census_from_hdf

CPU times: user 1min 9s, sys: 2.28 s, total: 1min 11s
Wall time: 1min 12s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254000000000,3254000000000,3254000000000,3254000000000,59.550114,Male,White,1960-09-12,3254000000006,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254000000001,3254000000001,3254000000001,3254000000000,58.674889,Female,White,1961-07-29,3254000000006,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254000000002,3254000000002,3254000000002,3254000000002,53.272919,Male,White,1966-12-23,3254000000007,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254000000003,3254000000003,3254000000003,3254000000002,54.643021,Female,White,1965-08-10,3254000000007,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254000000004,3254000000004,3254000000004,3254000000004,55.642822,Female,Black,1964-08-09,3254000000008,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15448912,3541000883506,3541000883506,3541000883506,3541000883504,7.739110,Male,White,2022-07-21,3541000618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448913,3541000883507,3541000883507,3541000883507,3541000883504,31.564119,Female,White,1998-09-23,3541000618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448914,3541000883508,3541000883508,3541000883508,3541000883508,27.369617,Male,Latino,2002-12-03,3541000618379,37,...,old silo rd,NaN,mooresville,1780,old silo rd,NaN,0,NC,mooresville,27217
15448915,3541000883509,3541000883509,3541000883509,3541000883509,71.134048,Male,Asian,1959-02-27,3541000618380,6,...,blue rdge ave,NaN,visalia,565,blue rdge ave,NaN,0,CA,visalia,91770


# Not only is reading in the HDF faster, but it automatically gives me the correct dtypes without me having to fiddle around with the `dtype` option in `read_csv` or having to convert things after loading the file

Note that the zipcodes are still malformatted in this version, i.e., they're missing leading zeros even though they're stored as strings instead of ints. We could fix this before saving the `.hdf` if desired.

In [125]:
%%time
df_census_from_hdf.equals(df_census_raw)

CPU times: user 2.5 s, sys: 468 ms, total: 2.97 s
Wall time: 2.97 s


True

# Load and save the WIC data for later use

Filesize reduction for csv -> hdf is 71M -> 59M for WIC. Similar ratio to what we got for census, but not quite as good for the smaller file.

In [129]:
%%time
df_wic_raw = datasets.load_data(
    f'{output_dir}/wic_observer.csv.bz2',
    convert_str_ids=True
)
df_wic_raw

CPU times: user 42.9 s, sys: 539 ms, total: 43.4 s
Wall time: 43.4 s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,7359000000037,7359000000037,7359000000037,7359000000036,21.328291,Female,White,1999-08-12,7359000000022,39,...,navy,NaN,struthers,37,navy,NaN,0,OH,struthers,45656
1,7359000000534,7359000000534,7359000000534,7359000000534,28.912081,Female,Latino,1992-01-11,7359000000222,48,...,lynbrook road,NaN,midland,1135,lynbrook road,NaN,0,TX,midland,77901
2,7359000000587,7359000000587,7359000000587,7359000000587,39.129105,Female,White,1981-10-23,7359000000238,6,...,almeda rd,NaN,berkeley,3631,almeda rd,NaN,0,CA,berkeley,91101
3,7359000000668,7359000000668,7359000000668,7359000000668,28.240271,Female,Latino,1992-09-12,7359000000266,6,...,fremont,NaN,murray,5900,fremont,NaN,0,CA,murray,93702
4,7359000001174,7359000001174,7359000001174,7359000001174,24.146772,Female,White,1996-10-16,7359000000476,17,...,brickett st,number 532,niles,1343,brickett st,number 532,0,IL,niles,60625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117386,5020000880494,5020000880494,5020000880494,5020000564117,0.028075,Female,Black,2030-01-13,5020000459053,39,...,james manor st,NaN,toledo,2640,james manor st,NaN,0,OH,toledo,44281
1117387,5020000880496,5020000880496,5020000880496,5020000595381,0.044600,Female,Latino,2030-01-07,5020000529146,35,...,huckleberry rd,NaN,albuquerque,10178,huckleberry rd,NaN,0,NM,albuquerque,87110
1117388,5020000880497,5020000880497,5020000880497,5020000618655,0.074165,Male,Latino,2029-12-27,5020000603095,6,...,forest ave,NaN,la verne,821,forest ave,NaN,0,CA,la verne,92101
1117389,5020000880499,5020000880499,5020000880499,5020000658640,0.031057,Female,White,2030-01-12,5020000446941,21,...,raleigh st,NaN,louisville,3636,raleigh st,NaN,0,KY,louisville,40160


In [130]:
df_wic_raw.dtypes

simulant_id                         int64
first_name_id                       int64
middle_name_id                      int64
last_name_id                        int64
age                               float32
sex                              category
race_ethnicity                   category
date_of_birth                    category
address_id                          int64
state_id                             int8
po_box                           category
puma                                int16
guardian_1                          int32
guardian_2                          int32
guardian_1_address_id             float32
guardian_2_address_id             float32
wic_year                         category
random_seed                          int8
year_of_birth                    category
state                            category
first_name                       category
middle_initial                   category
last_name                        category
zipcode                          c

In [131]:
sizemb(df_wic_raw)

206.875057

In [133]:
%%time
df_wic_raw.to_hdf(
    'wic_observer_2023_02_27_10_25_20.hdf', 'wic',
    format='table',
    complevel=5, complib='bzip2'
)

!ls -halt

total 1.9G
-rw-rw-r-- 1 ndbs IHME-users  59M Mar  7 17:40 wic_observer_2023_02_27_10_25_20.hdf
drwxrwsr-x 3 ndbs IHME-users 3.5K Mar  7 17:40 .
-rw-rw-r-- 1 ndbs IHME-users 235K Mar  7 17:39 2023_03_06_generate_wic_and_census_data.ipynb
-rw-rw-r-- 1 ndbs IHME-users 554M Mar  7 15:43 census_categoical_and_int_2.hdf
-rw-rw-r-- 1 ndbs IHME-users 615M Mar  6 23:07 census_categoical_and_int_1.hdf
-rw-rw-r-- 1 ndbs IHME-users 634M Mar  6 17:09 census_categoical_1.hdf
drwxrwsr-x 5 ndbs IHME-users 6.5K Mar  6 11:30 ..
drwxrwsr-x 2 ndbs IHME-users 1.0K Mar  6 11:19 .ipynb_checkpoints
-rw-rw-r-- 1 ndbs IHME-users 1.5K Mar  3 16:14 2023_03_03b_prep_wic_and_census_for_splink.ipynb
CPU times: user 17.1 s, sys: 525 ms, total: 17.6 s
Wall time: 18.7 s


# Test code after moving it to a new module

In [134]:
from vivarium_research_prl import datatypes

In [135]:
df_wic2 = datatypes.load_csv_data(
    f'{output_dir}/wic_observer.csv.bz2',
    convert_str_ids=True
)
df_wic2

,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,7359000000037,7359000000037,7359000000037,7359000000036,21.328291,Female,White,1999-08-12,7359000000022,39,...,navy,NaN,struthers,37,navy,NaN,0,OH,struthers,45656
1,7359000000534,7359000000534,7359000000534,7359000000534,28.912081,Female,Latino,1992-01-11,7359000000222,48,...,lynbrook road,NaN,midland,1135,lynbrook road,NaN,0,TX,midland,77901
2,7359000000587,7359000000587,7359000000587,7359000000587,39.129105,Female,White,1981-10-23,7359000000238,6,...,almeda rd,NaN,berkeley,3631,almeda rd,NaN,0,CA,berkeley,91101
3,7359000000668,7359000000668,7359000000668,7359000000668,28.240271,Female,Latino,1992-09-12,7359000000266,6,...,fremont,NaN,murray,5900,fremont,NaN,0,CA,murray,93702
4,7359000001174,7359000001174,7359000001174,7359000001174,24.146772,Female,White,1996-10-16,7359000000476,17,...,brickett st,number 532,niles,1343,brickett st,number 532,0,IL,niles,60625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117386,5020000880494,5020000880494,5020000880494,5020000564117,0.028075,Female,Black,2030-01-13,5020000459053,39,...,james manor st,NaN,toledo,2640,james manor st,NaN,0,OH,toledo,44281
1117387,5020000880496,5020000880496,5020000880496,5020000595381,0.044600,Female,Latino,2030-01-07,5020000529146,35,...,huckleberry rd,NaN,albuquerque,10178,huckleberry rd,NaN,0,NM,albuquerque,87110
1117388,5020000880497,5020000880497,5020000880497,5020000618655,0.074165,Male,Latino,2029-12-27,5020000603095,6,...,forest ave,NaN,la verne,821,forest ave,NaN,0,CA,la verne,92101
1117389,5020000880499,5020000880499,5020000880499,5020000658640,0.031057,Female,White,2030-01-12,5020000446941,21,...,raleigh st,NaN,louisville,3636,raleigh st,NaN,0,KY,louisville,40160


In [136]:
df_wic2.equals(df_wic_raw)

True